# Functions and imports

In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re#
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python/INPUT_FILE.csv --types_file data_python/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
         --miss_percentage_train 0 --miss_percentage_test 0 \
        --true_miss_file data_python/MISS_FILE --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [2]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/HI-VAE'

In [3]:
sample_size=178
# get file list
files=[i for i in os.listdir('data_python/') if not '_type' in i and not '_missing' in i and i not in '.DS_Store']
files.sort()
files

['ARObjectFinding.csv',
 'ARPlaceAndFindTelemetryVariance.csv',
 'ARScreenButtonPresses.csv',
 'BITDOTMotorInstructionReadingTimeRatios.csv',
 'MMSE_Attention_Concentration.csv',
 'MMSE_Language.csv',
 'MMSE_Memory_Recall.csv',
 'MMSE_Orientation.csv',
 'MMSE_Working_Memory_Registration.csv',
 'MotorDrawingFeatures.csv',
 'MotorTappingFeatures.csv',
 'MotorTestDurations.csv']

In [4]:
best_hyper = pd.read_csv('best_hyper_ALTOIDA_processed.csv')
best_hyper

,lrates,nbatch,wdecay,ydims,files,loss,sdims
0,0.001,16,0.001,1,ARObjectFinding.csv,24.957258,1
1,0.001,16,0.010,1,ARPlaceAndFindTelemetryVariance.csv,-290.294244,1
2,0.010,16,0.000,1,ARScreenButtonPresses.csv,22.558093,1
3,0.010,16,0.000,1,BITDOTMotorInstructionReadingTimeRatios.csv,4.106932,1
4,0.010,32,0.010,1,MMSE_Attention_Concentration.csv,0.910142,4
5,0.010,32,0.010,1,MMSE_Language.csv,0.613249,2
6,0.001,16,0.001,1,MMSE_Memory_Recall.csv,1.080130,3
7,0.010,32,0.000,1,MMSE_Orientation.csv,0.688555,4
8,0.010,32,0.001,1,MMSE_Working_Memory_Registration.csv,0.058547,1
9,0.010,16,0.001,1,MotorDrawingFeatures.csv,68.877012,1


# VP decoding

Run after bnet.R

In [5]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/HI-VAE'

In [6]:
meta = pd.read_csv('metaenc.csv')
meta

,scode_ARObjectFinding,SUBJID,zcode_ARObjectFinding,scode_ARPlaceAndFindTelemetryVariance,zcode_ARPlaceAndFindTelemetryVariance,scode_ARScreenButtonPresses,zcode_ARScreenButtonPresses,scode_BITDOTMotorInstructionReadingTimeRatios,zcode_BITDOTMotorInstructionReadingTimeRatios,scode_MMSE_Attention_Concentration,...,scode_MMSE_Orientation,zcode_MMSE_Orientation,scode_MMSE_Working_Memory_Registration,zcode_MMSE_Working_Memory_Registration,scode_MotorDrawingFeatures,zcode_MotorDrawingFeatures,scode_MotorTappingFeatures,zcode_MotorTappingFeatures,scode_MotorTestDurations,zcode_MotorTestDurations
0,0,009-20_001_1,-1.042224,0,0.935868,0,0.585778,0,0.680546,3,...,3,-0.916167,0,1.523856,0,0.988264,0,-3.996608,0,1.106418
1,0,009-20_002_1,0.128795,0,2.264648,0,-0.106377,0,-0.517770,2,...,1,10.482780,0,1.523856,0,0.364599,0,0.016033,0,0.654828
2,0,009-20_003_1,1.216998,0,1.235406,0,0.070480,0,0.689196,2,...,1,10.482780,0,1.523856,0,0.361916,0,0.629590,0,-1.276689
3,0,009-20_004_1,0.270090,0,2.717478,0,-0.446318,0,-0.118558,2,...,3,-0.916167,0,1.523856,0,-0.080454,0,1.088204,0,-0.612896
4,0,009-20_005_1,-0.627163,0,2.300178,0,0.156115,0,0.362346,2,...,0,3.068466,0,0.961632,0,-0.087803,0,0.182084,0,-0.426074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0,005_012_2,2.632748,0,3.557675,0,-0.353706,0,-1.246386,3,...,1,10.482780,0,1.523856,0,-0.401649,0,-1.877998,0,0.281586
174,0,005_012_3,2.256792,0,3.426479,0,-1.651626,0,-1.246386,3,...,1,10.482780,0,1.523856,0,-0.401649,0,-1.877998,0,0.281586
175,0,005_015_1,2.004861,0,3.820880,0,-0.348762,0,-1.170140,3,...,0,3.068466,0,1.523856,0,-0.028437,0,-0.521731,0,-0.310235
176,0,005_015_2,2.142821,0,2.720714,0,-1.004784,0,-1.170140,3,...,0,3.068466,0,1.523856,0,-0.028437,0,-0.521731,0,-0.310235


In [7]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/data_out/main_VirtualPPts.csv')
VPcodes

,zcode_ARObjectFinding,zcode_ARPlaceAndFindTelemetryVariance,zcode_ARScreenButtonPresses,zcode_BITDOTMotorInstructionReadingTimeRatios,scode_MMSE_Attention_Concentration,zcode_MMSE_Attention_Concentration,scode_MMSE_Language,zcode_MMSE_Language,scode_MMSE_Memory_Recall,zcode_MMSE_Memory_Recall,...,SA_yearsOfEducation_VIS1,SA_DX_VIS1,SA_Amyloid_VIS1,AUX_SA_DX_VIS1,AUX_MMSE_Attention_Concentration_VIS1,AUX_MMSE_Language_VIS1,AUX_MMSE_Memory_Recall_VIS1,AUX_MMSE_Orientation_VIS1,AUX_MMSE_Working_Memory_Registration_VIS1,visitmiss_VIS1
0,-0.909795,1.050856,0.761325,1.380295,3,1.154054,1,0.356885,2,-0.786465,...,7.770544,4,1,0,1,1,0,1,0,0
1,0.898027,4.643477,-0.613027,-0.135637,2,7.248290,0,-7.274332,2,-0.557504,...,10.520668,3,1,0,0,0,0,0,0,0
2,0.521211,2.734121,-0.460279,-1.438894,3,-1.257662,0,-7.298436,2,-0.657445,...,15.946635,2,1,0,1,0,1,0,1,0
3,0.612674,3.409595,0.419893,0.175197,3,-0.913665,0,-7.545495,2,-1.023005,...,8.929382,3,1,0,0,0,0,0,0,0
4,2.378629,4.408084,0.590558,-0.278093,3,-0.146857,0,-7.410556,2,-0.421779,...,13.935964,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2.667322,3.198189,-0.518622,0.111390,3,0.475321,0,-6.769284,0,4.130582,...,16.173824,0,0,0,1,0,0,0,1,0
174,1.180671,3.780643,-0.556731,-0.275301,2,7.248290,0,-7.124229,2,-2.006194,...,13.816822,0,0,0,0,0,1,0,0,0
175,2.945121,4.078771,-1.131086,0.343885,3,-0.654535,0,-7.131695,0,4.147564,...,14.868861,0,0,0,1,0,0,0,1,0
176,1.043313,2.574058,-0.006605,0.163075,2,7.248290,0,-7.097291,0,4.094846,...,13.943636,1,0,0,0,0,0,0,0,0


In [ ]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/data_out/main_VirtualPPts.csv')

dfs=list()
virt=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    dec=helpers.dec_network(settings,zcodes,scodes,VP=True)
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_cols.csv')['x']
    dat=pd.DataFrame(dec)
    dat.columns=names
    dat['SUBJID']=subj
    virt.append(dec)
    dfs.append(dat)

virt_dic=dict(zip(files,virt))
decoded=helpers.merge_dat(dfs)
decoded.to_csv('decodedVP.csv',index=False)

Get Loglikelihoods for R plot!

In [8]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/data_out/main_VirtualPPts.csv')

dfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    loglik=helpers.dec_network_loglik(settings,zcodes,scodes,VP=True)
    loglik=np.nanmean(np.array(loglik).T,axis=1)
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    dat=pd.DataFrame(loglik)
    dat.columns=[f]
    dat['SUBJID']=subj
    dfs.append(dat)

decoded=helpers.merge_dat(dfs)
decoded.to_csv('virtual_logliks.csv',index=False)

Namespace(batch_size=178, data_file='data_python/ARObjectFinding.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARObjectFinding', true_miss_file='data_python/ARObjectFinding_missing.csv', types_file='data_python/ARObjectFinding_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders




/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Encoder...
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


[*] Defining Decoder...
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Th

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python/ARPlaceAndFindTelemetryVariance.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARPlaceAndFindTelemetryVariance', true_miss_file='data_python/ARPlaceAndFindTelemetryVariance_missing.csv', types_file='data_python/ARPlaceAndFindTelemetryVariance_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARPlaceAndFindTelemetryVariance/ARPlaceAndFindTelemetryVariance.ckpt
Model restored: ./Saved_Networks/ARPlaceAndFindTelemetryVariance/ARPlaceAndFindTelemetryVariance.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/ARPlaceAndFindTelemet

...

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorDrawingFeatures/MotorDrawingFeatures.ckpt
Model restored: ./Saved_Networks/MotorDrawingFeatures/MotorDrawingFeatures.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorDrawingFeatures_vpmiss.csv


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python/MotorTappingFeatures.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='MotorTappingFeatures', true_miss_file='data_python/MotorTappingFeatures_missing.csv', types_file='data_python/MotorTappingFeatures_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTappingFeatures/MotorTappingFeatures.ckpt
Model restored: ./Saved_Networks/MotorTappingFeatures/MotorTappingFeatures.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTappingFeatures_vpmiss.csv


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python/MotorTestDurations.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='MotorTestDurations', true_miss_file='data_python/MotorTestDurations_missing.csv', types_file='data_python/MotorTestDurations_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
Model restored: ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTestDurations_vpmiss.csv
